In [1]:
import pandas as pd

file_path = r"C:\Users\aryan\Downloads\twitter_training.csv"  # Make sure this is the correct path to your file
df = pd.read_csv(file_path, header=None)  # Read without header
df.columns = ['id', 'game', 'sentiment', 'text'] # Rename columns
print(df.head())
print(df.info())
print(df['text'].sample(10))

     id         game sentiment  \
0  2401  Borderlands  Positive   
1  2401  Borderlands  Positive   
2  2401  Borderlands  Positive   
3  2401  Borderlands  Positive   
4  2401  Borderlands  Positive   

                                                text  
0  im getting on borderlands and i will murder yo...  
1  I am coming to the borders and I will kill you...  
2  im getting on borderlands and i will kill you ...  
3  im coming on borderlands and i will murder you...  
4  im getting on borderlands 2 and i will murder ...  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74682 entries, 0 to 74681
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         74682 non-null  int64 
 1   game       74682 non-null  object
 2   sentiment  74682 non-null  object
 3   text       73996 non-null  object
dtypes: int64(1), object(3)
memory usage: 2.3+ MB
None
12698              damn.. i dont think its worth the grind
41694    

In [2]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np # Import numpy to check for NaN values

nltk.download('stopwords')
nltk.download('wordnet')

stop = set(stopwords.words('english'))
lemm = WordNetLemmatizer()

def clean_text(txt):
    # Since missing values are handled before calling this function,
    # we can assume the input is a string or has been converted to string representation by fillna and astype(str).
    # We can keep the isinstance check as an extra safeguard.
    if not isinstance(txt, str):
        return ""
    txt = txt.lower()
    txt = re.sub(r'http\S+','', txt)
    txt = re.sub(r'@\w+','', txt)
    txt = re.sub(r'#','', txt)
    txt = re.sub(r'[^a-z0-9\s]','', txt)
    tokens = txt.split()
    tokens = [lemm.lemmatize(t) for t in tokens if t not in stop]
    return " ".join(tokens)

# Fill missing values with empty strings and then convert the column to string type
df['cleaned'] = df['text'].fillna('').astype(str).apply(clean_text)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\aryan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\aryan\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
from transformers import pipeline
sentiment_pipeline = pipeline("sentiment-analysis")
res = sentiment_pipeline(df['cleaned'].tolist())
# Parse ’label’ and ’score’ etc


C:\Users\aryan\miniconda3\envs\sentiment\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Bar chart of overall sentiment labels
sns.countplot(data=df, x='sentiment_label')
plt.title("Overall sentiment distribution")
plt.show()

# Line chart of daily average sentiment
# plt.figure(figsize=(12,6)) # Commented out as 'daily' variable is not created
# sns.lineplot(data=daily, x='date', y='sentiment_score') # Commented out as 'daily' variable is not created
# plt.title("Daily average sentiment score over time") # Commented out as 'daily' variable is not created
# plt.show() # Commented out as 'daily' variable is not created

# Heatmap for brand vs sentiment label
plt.figure(figsize=(10,8))
# Using 'game' as a proxy for 'brand' as used in the previous cell
sns.heatmap(brand_sent, annot=True, fmt=".2f", cmap="coolwarm")
plt.title("Sentiment distribution by game") # Updated title to reflect 'game'
plt.show()

##Summary

* Data Loading and Cleaning: The notebook loads data from a CSV file, renames columns, and performs text cleaning by removing URLs, mentions, hashtags, and special characters. It also handles missing text values by replacing them with empty strings.
Sentiment Analysis using VADER: The VADER sentiment analysis tool is used to calculate a compound sentiment score for each cleaned text entry. These scores are then categorized into 'positive', 'negative', or 'neutral' labels based on predefined thresholds.

 * Sentiment Distribution by Game: The analysis then calculates the distribution of sentiment labels for each game in the dataset, providing insights into how positively or negatively each game is perceived in the text data.

* Visualization: The notebook includes visualizations to show the overall sentiment distribution and a heatmap illustrating the sentiment distribution across different games. The heatmap of sentiment distribution by game is particularly insightful, as it shows the proportion of positive, neutral, and negative sentiment for each game